## Imports

In [33]:
# pip install --upgrade scikit-learn

In [34]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.0.3
Sklearn  0.23.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [35]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "lorenzodemiri"
    os.environ['KAGGLE_KEY']      = "5620bc6761eb9d35f4d9e91c9b47af78"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../datasets/titanic/"

## Load data

In [36]:
df = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [37]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [38]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1:
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [39]:
def get_Title_from_Name(string):
    string_ret = string.split(", ")[1].split(".")[0]
    return string_ret

df["Title"]      = df['Name'].map(get_Title_from_Name)
df_test["Title"] = df_test['Name'].map(get_Title_from_Name)

# Exercise 2:
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [40]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [41]:
# CODE HERE

df["Title"] = df["Title"].replace(title_dictionary) 
df_test["Title"] = df_test["Title"].replace(title_dictionary)
df_test["Title"] 

PassengerId
892         Mr
893        Mrs
894         Mr
895         Mr
896        Mrs
         ...  
1305        Mr
1306      Dona
1307        Mr
1308        Mr
1309    Master
Name: Title, Length: 418, dtype: object

# Exercise BONUS:
Try to extract some information from the feature **Ticket**

# Exercise BONUS:
Try to extract some information from the feature **Cabin**

# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [42]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [43]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3:
Create a **ColumnTransformer for Tree Models**. Remember:
- Categorical: Some SimpleImputer -> Some Encoder
- Numerical: Some SimpleImputer -> NO Encoder

In [44]:
num_preprocessing = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)),
  ('scaler',  preprocessing.StandardScaler())
])

cat_preporcessing = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant',
  fill_value='missing')),
  # Some Encoder here. Remember to handle_unknown
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [45]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

ModuleNotFoundError: No module named 'lightgbm'

In [22]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier(),
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers['LightGBM']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5:
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [24]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    train_size=1/5,
    random_state=0
)
model_name_vec = []
accurancy = []
bal_accurancy = []
for model_name, model in tree_classifiers.items():
    tree_model = tree_classifiers[model_name].fit(x_train, y_train)
    y_pred = tree_model.predict(x_val)
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    model_name_vec.append(model_name)
    accurancy.append(metrics.accuracy_score(y_pred,y_val))
    bal_accurancy.append(metrics.balanced_accuracy_score(y_pred, y_val))

df_models = pd.DataFrame(data=model_name_vec, columns=['model_name'])
df_models['accurancy'] = accurancy
df_models['bal_accurancy'] = bal_accurancy
df_models

1:	learn: 0.6857553	total: 1.67ms	remaining: 834ms
2:	learn: 0.6825949	total: 2.38ms	remaining: 790ms
3:	learn: 0.6792392	total: 3.08ms	remaining: 768ms
4:	learn: 0.6759953	total: 3.78ms	remaining: 752ms
5:	learn: 0.6720536	total: 4.54ms	remaining: 752ms
6:	learn: 0.6685321	total: 5.23ms	remaining: 742ms
7:	learn: 0.6648876	total: 5.93ms	remaining: 736ms
8:	learn: 0.6613480	total: 6.64ms	remaining: 731ms
9:	learn: 0.6587387	total: 7.37ms	remaining: 730ms
10:	learn: 0.6551970	total: 8.13ms	remaining: 731ms
11:	learn: 0.6518338	total: 8.84ms	remaining: 728ms
12:	learn: 0.6489940	total: 9.55ms	remaining: 725ms
13:	learn: 0.6456995	total: 10.4ms	remaining: 732ms
14:	learn: 0.6424109	total: 11.2ms	remaining: 733ms
15:	learn: 0.6406721	total: 11.6ms	remaining: 714ms
16:	learn: 0.6386928	total: 11.9ms	remaining: 688ms
17:	learn: 0.6357968	total: 71.9ms	remaining: 3.92s
18:	learn: 0.6341290	total: 72.3ms	remaining: 3.73s
19:	learn: 0.6316962	total: 73ms	remaining: 3.58s
20:	learn: 0.6290649	to

,model_name,accurancy,bal_accurancy
0,Decision Tree,0.737728,0.722078
1,Extra Trees,0.771388,0.758313
2,Random Forest,0.772791,0.759930
3,AdaBoost,0.754558,0.737850
4,Skl GBM,0.781206,0.765781
5,Skl HistGBM,0.761571,0.749358
6,XGBoost,0.769986,0.755979
7,LightGBM,0.765778,0.755206
8,CatBoost,0.795231,0.780266


# Exercise 6:
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [28]:
skf = model_selection.StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=0
)
accurancy = []
model_name_vec = []
for model_name, model in tree_classifiers.items():
    cros = model_selection.cross_val_predict(tree_classifiers[model_name], x_train, y_train, cv=skf)
    accurancy.append(model.score(x_val, y_val))
    model_name_vec.append(model_name)
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
df_cross = pd.DataFrame(data=model_name_vec, columns=['Models'])
df_cross['Accurancy'] = accurancy
df_cross

1:	learn: 0.6867077	total: 2.36ms	remaining: 1.18s
2:	learn: 0.6838641	total: 3.29ms	remaining: 1.09s
3:	learn: 0.6821449	total: 3.93ms	remaining: 978ms
4:	learn: 0.6793416	total: 4.73ms	remaining: 942ms
5:	learn: 0.6761152	total: 5.6ms	remaining: 928ms
6:	learn: 0.6728070	total: 6.38ms	remaining: 905ms
7:	learn: 0.6702223	total: 7ms	remaining: 868ms
8:	learn: 0.6673614	total: 7.8ms	remaining: 859ms
9:	learn: 0.6646812	total: 8.68ms	remaining: 860ms
10:	learn: 0.6623001	total: 9.35ms	remaining: 841ms
11:	learn: 0.6589717	total: 41.7ms	remaining: 3.44s
12:	learn: 0.6560941	total: 42.9ms	remaining: 3.26s
13:	learn: 0.6527985	total: 44ms	remaining: 3.1s
14:	learn: 0.6500431	total: 45.2ms	remaining: 2.97s
15:	learn: 0.6472971	total: 46.4ms	remaining: 2.85s
16:	learn: 0.6455545	total: 47ms	remaining: 2.72s
17:	learn: 0.6435620	total: 47.8ms	remaining: 2.61s
18:	learn: 0.6409480	total: 48.7ms	remaining: 2.51s
19:	learn: 0.6390706	total: 49.5ms	remaining: 2.42s
20:	learn: 0.6366133	total: 50.

,Models,Accurancy
0,Decision Tree,0.737728
1,Extra Trees,0.771388
2,Random Forest,0.772791
3,AdaBoost,0.754558
4,Skl GBM,0.781206
5,Skl HistGBM,0.761571
6,XGBoost,0.769986
7,LightGBM,0.765778
8,CatBoost,0.795231


# Exercise 7
Train **with all data** the best model

In [31]:
best_model = 'CatBoost'
# Train with all data your best model
tree_best = tree_classifiers[best_model].fit(x_train,y_train)

1:	learn: 0.6857553	total: 1.49ms	remaining: 741ms
2:	learn: 0.6825949	total: 2.18ms	remaining: 724ms
3:	learn: 0.6792392	total: 2.87ms	remaining: 714ms
4:	learn: 0.6759953	total: 3.61ms	remaining: 718ms
5:	learn: 0.6720536	total: 4.27ms	remaining: 707ms
6:	learn: 0.6685321	total: 4.93ms	remaining: 700ms
7:	learn: 0.6648876	total: 5.58ms	remaining: 692ms
8:	learn: 0.6613480	total: 6.25ms	remaining: 688ms
9:	learn: 0.6587387	total: 9.96ms	remaining: 986ms
10:	learn: 0.6551970	total: 11.7ms	remaining: 1.05s
11:	learn: 0.6518338	total: 13.3ms	remaining: 1.09s
12:	learn: 0.6489940	total: 14.8ms	remaining: 1.13s
13:	learn: 0.6456995	total: 19ms	remaining: 1.34s
14:	learn: 0.6424109	total: 20.4ms	remaining: 1.34s
15:	learn: 0.6406721	total: 27.8ms	remaining: 1.71s
16:	learn: 0.6386928	total: 28.4ms	remaining: 1.64s
17:	learn: 0.6357968	total: 29.2ms	remaining: 1.59s
18:	learn: 0.6341290	total: 29.6ms	remaining: 1.53s
19:	learn: 0.6316962	total: 30.8ms	remaining: 1.51s
20:	learn: 0.6290649	to

# Exercise 8
With your best model, generate the predicitions for test data (x_test)

In [32]:
test_pred = tree_best.predict(x_test)

# Exercise 9

Submit to kaggle using the kaggle API. And send us your score. You can try to improve it.

In [33]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [34]:
sub.to_csv("sub.csv")

In [35]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100%|██████████████████████████████████████| 2.77k/2.77k [00:00<00:00, 6.24kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

# Exercise BONUS

Knowing how to export your models is very important for putting models in production. Try to
- Export and Load the ColumTransformer in pickle
- Export and Load the ColumTransformer in joblib
- Export and load the Pipeline